Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 2.71 ms, sys: 3.96 ms, total: 6.68 ms
Wall time: 10.4 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7082/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 288 ms, sys: 33.9 ms, total: 322 ms
Wall time: 2.08 s


[1.1864448072334564,
 1.0941295575150698,
 1.3370202098286268,
 1.2689508659599413,
 1.9788939691922738,
 1.9641607286784961,
 1.6962248017108323,
 1.0772462163987553,
 1.9443861154107551,
 1.3475727603947651,
 1.1052040540018744,
 1.891695481764902,
 1.4688311093249402,
 1.2830830213829976,
 1.754913903244228,
 1.0076771515876195,
 1.859331276396296,
 1.5170010432019962,
 1.4052709178962672,
 1.27684172641569,
 1.8156512516209995,
 1.1982357766394216,
 1.8609761549611017,
 1.2822477848429574,
 1.2446655881437316,
 1.1697736572593422,
 1.4140610588072844,
 1.48286109159896,
 1.3013809708565303,
 1.0055213345564915,
 1.0039728351071489,
 1.7005453732494082,
 1.5425806747052833,
 1.4982953782276358,
 1.6627463735729533,
 1.2122661190676187,
 1.8289877896656868,
 1.5371571071076948,
 1.2698383312716375,
 1.9787766438445789,
 1.1496674664316693,
 1.0159956999174045,
 1.0604927943467608,
 1.3688166207834689,
 1.3020524288445703,
 1.5529851732036222,
 1.2480059974386624,
 1.9756544687866544,

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.550716446839449,
 1.3606043508072934,
 1.9628612496666216,
 1.0149009615175397,
 1.8543114852488192,
 1.9737167794321753,
 1.2198679059100224,
 1.7220963439209243,
 1.0900455860385434,
 1.1002601194011015,
 1.219015257343086,
 1.351494317637837,
 1.2439271403100678,
 1.0042673064959504,
 1.1794876427343606,
 1.9311401889406696,
 1.0760723289760619,
 1.5748017719320047,
 1.0559100392894327,
 1.314685095334184,
 1.2723356817855624,
 1.8789140180181427,
 1.9526233371634663,
 1.9251241147479634,
 1.8203844514820662,
 1.1523458925512213,
 1.1640980065220994,
 1.0418301663721863,
 1.2040461936216822,
 1.9041583713054677,
 1.8609823439110458,
 1.4244408873110308,
 1.5637425424969906,
 1.3233191590296753,
 1.1870795148492759,
 1.481262465448973,
 1.635861648884306,
 1.9816217902430995,
 1.449098914163371,
 1.1477953300495374,
 1.2343315179540717,
 1.7586133767215755,
 1.8327811520349009,
 1.8619427021186186,
 1.2393222874732892,
 1.7327083365078293,
 1.8643111552030183,
 1.7335678841562787,

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 288 ms, sys: 15.4 ms, total: 303 ms
Wall time: 2.61 s
